**1.1** Import main modules

We will first import the main modules

In [ ]:
import pandas as pd
import numpy as np
import datetime
pd.options.display.max_columns = 100

**1.2**	Establish random data range (Policy Counts, Years, Dates)

Now to set the breadth of data we are going to work with

In [ ]:
"""Define parameters of data"""
# Number of entries
PolicyCount = 15000
# 9-year period
YearEndCap = 2017
YearStartCap = 2007
# Dates
DateEndCap = datetime.date(YearEndCap, 12, 31)    # year, month, day
DateStartCap = datetime.date(YearStartCap, 12, 31)  # year, month, day

**1.3** Other Assumptions

-No admin costs, claims paid in bulk on transaction date rather than multiple dates for those with multiple claims, ignoring earned gross premium calculations etc

* **2. Create Randomized Claims Data**

Now we will use randomisation with probabilistic distributions to simulate some claims data. First we establish the initial data-frame.

In [ ]:
"""Create Main DataFrame filled with NaN's"""
# Establish initial data-frame
columns_1 = ['Insured_ID', 'Insured_Date', 'Claims_Number', 'Claims_Amount', 'Transaction_Date',
           'Insured_Year', 'Insured_Quarter',
           'Transaction_Year', 'Transaction_Quarter']
ClaimsData = pd.DataFrame(columns=columns_1)

**2.1** Insured IDs

This is just for labelling purposes to simulate some reality to this. Do note that I have hidden some outputs to neaten up this Kernel. *Unhide to view output

In [ ]:
# Insured_ID's
ClaimsData['Insured_ID'] = list(range(1, PolicyCount+1))
print(ClaimsData['Insured_ID'])

0            1
1            2
2            3
3            4
4            5
         ...  
14995    14996
14996    14997
14997    14998
14998    14999
14999    15000
Name: Insured_ID, Length: 15000, dtype: int64


**2.2** Insured Dates

This is the date where the insured signs for their policy. Here we will use a random selection of dates between our date range. *Unhide to view output

In [ ]:
# Insured_Date's
# Random distribution
import random
for row in range(0, PolicyCount):
    n_days = (DateEndCap-DateStartCap).days
    random_days = random.randint(0, n_days-1)
    Random_Insured_Date = DateStartCap + datetime.timedelta(days=1) + datetime.timedelta(days=random_days)
    ClaimsData.loc[row, 'Insured_Date'] = Random_Insured_Date
print(ClaimsData['Insured_Date'])

0        2014-03-08
1        2014-11-28
2        2014-02-01
3        2009-03-27
4        2008-10-17
            ...    
14995    2014-08-11
14996    2009-03-29
14997    2016-10-08
14998    2013-09-22
14999    2008-07-30
Name: Insured_Date, Length: 15000, dtype: object


**2.3** Claims Numbers

We will now simulate the claim numbers using randomised poisson distribution. In real world scenarios, this is the most common distribution for claim numbers. *Unhide to view output

In [ ]:
# Claims_Number's
# Poisson random distribution
# Poisson parameters
Lambda = 10
Size = 1
for row in range(0, PolicyCount):
    ClaimCount = np.random.poisson(1, 1)
    ClaimsData.loc[row, 'Claims_Number'] = ClaimCount

# Remove the square brackets (i.e.a list within a list) by passing into a list & back into df again
ClaimsData['Claims_Number'] = pd.DataFrame(ClaimsData['Claims_Number'].values.tolist())
print(ClaimsData['Claims_Number'])

0        0
1        0
2        1
3        0
4        1
        ..
14995    0
14996    2
14997    1
14998    2
14999    0
Name: Claims_Number, Length: 15000, dtype: int64


**2.4** Claim Amounts

Similarly, for claim amounts we will also use randomisation but with Log Normal distribution instead. Do note that here we use a nested loop referncing code. Where if the random claim number was 0 we will have 0 claim amount. For cases without 0 random claim numbers, we will respectively generate and sum 'n' number of random Log Normal distributed claim amount. E.g. Claim Number 1 will have 1 random Log Normal distributed amount, Claim Number 2 will have the sum of 2 random Log Normal distributed amount etc.

I have also included a spare repetitive Min Max function to simulate reinsurance if need. The code is a 'Do While' loop so as to not disrupt the distribution of claim amounts. But have excluded it in the case. 

*Unhide to view output

In [ ]:
"""Special Case if need to simulate claims amount minimum & maximum limit. E.g. Reinsurance cases XOL"""
import random
def trunc_amt(mu, sigma, bottom, top):
    a = random.lognormal(mu,sigma)
    while (bottom <= a <= top) == False:
        a = random.lognormal(mu,sigma)
    return a

In [ ]:
# Claims_Amount's
# Gaussian random distribution
# Gaussian parameters
MeanClaimAmt = 10
StdDevClaimAmt = 4
for row in range(0, PolicyCount):
    if ClaimsData.loc[row, 'Claims_Number'] == 0:
        # Impute 0 so that ClaimAmount is 0
        ClaimsData.loc[row, 'Claims_Amount'] = 0
    else:
        ClaimNumber = ClaimsData.loc[row, 'Claims_Number']
        num = np.random.lognormal(MeanClaimAmt, StdDevClaimAmt, ClaimNumber).sum()
        ClaimsData.loc[row, 'Claims_Amount'] = num

# Remove the square brackets (i.e.a list within a list) by passing into a list & back into df again
ClaimsData['Claims_Amount'] = pd.DataFrame(ClaimsData['Claims_Amount'].values.tolist())
print(ClaimsData['Claims_Amount'])

0        0.000000e+00
1        0.000000e+00
2        1.366208e+04
3        0.000000e+00
4        1.625353e+06
             ...     
14995    0.000000e+00
14996    4.249528e+05
14997    3.787425e+04
14998    2.931482e+05
14999    0.000000e+00
Name: Claims_Amount, Length: 15000, dtype: float64


**2.5** Transaction Dates

The transaction dates are the dates that the insurer paid to the insured for a claim made. Just as in claim amounts, we will do a nested loop referencing. Where if the claim number was 0 we will input Transaction Date as the Insured Date, to achieve a 0 lag year. While for cases without 0 as the claim numbers, we will generate a random date between the 'Insured Date' and the YearEndCap of out data range. *Unhide to view output

In [ ]:
# Transaction_Date's
# Random distribution
import random
for row in range(0, PolicyCount):
    DateStart = ClaimsData.loc[row, 'Insured_Date']
    if ClaimsData.loc[row, 'Claims_Number'] == 0:
        # Impute InsuredDate so that Lag(i.e.DevelopmentPeriod) will be 0
        ClaimsData.loc[row, 'Transaction_Date'] = DateStart
    elif (DateEndCap-DateStart).days <=0:
        ClaimsData.loc[row, 'Transaction_Date'] = DateStart
    else:
        n_days = (DateEndCap-DateStart).days
        random_days = random.randint(1, n_days) # Min 1 day to avoid conflict of zero days and no claims
        Random_Transaction_Date = DateStart + datetime.timedelta(days=random_days)
        ClaimsData.loc[row, 'Transaction_Date'] = Random_Transaction_Date
print(ClaimsData['Transaction_Date'])

0        2014-03-08
1        2014-11-28
2        2015-12-05
3        2009-03-27
4        2009-05-06
            ...    
14995    2014-08-11
14996    2017-12-08
14997    2017-04-22
14998    2017-03-26
14999    2008-07-30
Name: Transaction_Date, Length: 15000, dtype: object


* **3. Preview Raw Data (Part A)**

**3.1** Randomized Claims Data

Now to preview our random data that we have improvised so far.

In [ ]:
display(ClaimsData.head(10))

* **4. Preview Raw Data (Part A)**

**4.1** Extract Years or Quarters (Depending on your choice of lag period)

Now to prepare our calculation for the different development periods. I have chosen years as the development period here. You may use months alternatively. *Unhide to view output

*Do note that I will use lag year synonymously with development year here.

In [ ]:
# Extract & Impute Date Components
# Jan-Mar=1, Apr-Jun=2, July-Sep=3, Oct-Dec=4
# Insured Year
ClaimsData['Insured_Year'] = ClaimsData['Insured_Date'].apply(lambda x: x.year)
ClaimsData['Transaction_Year'] = ClaimsData['Transaction_Date'].apply(lambda x: x.year)
# Insured Month
ClaimsData['Insured_Quarter'] = ClaimsData['Insured_Date'].apply(lambda x: x.month)
ClaimsData['Transaction_Quarter'] = ClaimsData['Transaction_Date'].apply(lambda x: x.month)
print(ClaimsData[['Insured_Date', 'Insured_Year', 'Transaction_Date', 'Transaction_Year']])

      Insured_Date  Insured_Year Transaction_Date  Transaction_Year
0       2014-03-08          2014       2014-03-08              2014
1       2014-11-28          2014       2014-11-28              2014
2       2014-02-01          2014       2015-12-05              2015
3       2009-03-27          2009       2009-03-27              2009
4       2008-10-17          2008       2009-05-06              2009
...            ...           ...              ...               ...
14995   2014-08-11          2014       2014-08-11              2014
14996   2009-03-29          2009       2017-12-08              2017
14997   2016-10-08          2016       2017-04-22              2017
14998   2013-09-22          2013       2017-03-26              2017
14999   2008-07-30          2008       2008-07-30              2008

[15000 rows x 4 columns]


* **5. Calculations (Part A)**

**5.1**	Year Lags

This is pretty straightforward where we simply take the difference between the "Transaction year" and the "Insured year" columns. In simple terms, the number of years between the insured seured the policy and when a claim was received. *Unhide to view output

In [ ]:
# Year ONLY lag
ClaimsData['Year_Only_Lag'] = ClaimsData['Transaction_Year'] - ClaimsData['Insured_Year']
print(ClaimsData)

       Insured_ID Insured_Date  Claims_Number  Claims_Amount Transaction_Date  \
0               1   2014-03-08              0   0.000000e+00       2014-03-08   
1               2   2014-11-28              0   0.000000e+00       2014-11-28   
2               3   2014-02-01              1   1.366208e+04       2015-12-05   
3               4   2009-03-27              0   0.000000e+00       2009-03-27   
4               5   2008-10-17              1   1.625353e+06       2009-05-06   
...           ...          ...            ...            ...              ...   
14995       14996   2014-08-11              0   0.000000e+00       2014-08-11   
14996       14997   2009-03-29              2   4.249528e+05       2017-12-08   
14997       14998   2016-10-08              1   3.787425e+04       2017-04-22   
14998       14999   2013-09-22              2   2.931482e+05       2017-03-26   
14999       15000   2008-07-30              0   0.000000e+00       2008-07-30   

       Insured_Year  Insure

**5.2**	Compile Past Claims Data (Incremental  & Cumulative Amounts)

Now to simply compile the claims data in a sorted format. Specifically, in a default ascending insured year and lag year order. This does help greatly in terms of indexing later on. *Unhide to view output

Code Explanation-

Incremental - We are using the "Claim Amounts" as the output column. We then set filtering rows ("Insured year") and columns ("Lag year"), as we want to see the aggregate for each combination of "Insured year" and "Lag year". The "sum" function acts as a the aggregate function. Subsequently, the "reset_index" function is used to force the data-frame to use the default numerical indexing for the leftmost column. We then assign this to a new data-frame "py_data"

Cumulative - Just as before, we group the data-frame by the "Insured year" and output the corresponding "Claim Amounts". However, now we instead use the new data-frame "py_data" and we do not require the added columns "Lag year". In addition, we also now use the "cumsum" as a the aggregate function to derive the cumulative claim amounts for each.

In [ ]:
# Compile Past Claims Data
# Incremental Claims Amount
py_data = ClaimsData['Claims_Amount'].groupby([ClaimsData['Insured_Year'], ClaimsData['Year_Only_Lag']]).sum().reset_index()
# Convert into data-frame
py_data = pd.DataFrame(py_data)
# Cumulative Claims Amount
py_data["cumsum"] = py_data["Claims_Amount"].groupby(py_data["Insured_Year"]).cumsum()
print(py_data)

    Insured_Year  Year_Only_Lag  Claims_Amount        cumsum
0           2008              0   5.693138e+08  5.693138e+08
1           2008              1   5.032754e+08  1.072589e+09
2           2008              2   1.118322e+09  2.190911e+09
3           2008              3   6.433106e+08  2.834221e+09
4           2008              4   4.086228e+08  3.242844e+09
5           2008              5   3.803889e+09  7.046733e+09
6           2008              6   1.570847e+09  8.617580e+09
7           2008              7   1.757582e+09  1.037516e+10
8           2008              8   4.205037e+08  1.079567e+10
9           2008              9   2.715208e+09  1.351087e+10
10          2009              0   5.182929e+08  5.182929e+08
11          2009              1   1.936207e+10  1.988036e+10
12          2009              2   6.250441e+09  2.613080e+10
13          2009              3   2.921530e+09  2.905233e+10
14          2009              4   6.522494e+09  3.557483e+10
15          2009        

**Do note from this point onwards, we are dealing with Inflated Adjusted Chain Ladder (IACL) calculations. As mentioned before, I have separated the codings which are required for the IACL (with "Inflated" Headers) from those required for a basic chain ladder (with "Non Inflated" Headers)**

**5.3**	Establish Inflation Indexes

For further real world simulation, we will now use the approximated past UK Inflation rates.

In [ ]:
# Establish Inflation Index
# Create data-frame of Cumulative inflation rates
columns_2 = ['Year', 'CumPastInflation']
Inflation_df = pd.DataFrame(columns=columns_2)
# Past Inflation Years
Inflation_df['Year'] = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
# Past Inflation Index
Inflation_df['CumPastInflation'] = [1.32, 1.27, 1.28, 1.22, 1.16, 1.12, 1.09, 1.07, 1.05, 1.04, 1.00]
display(Inflation_df)

,Year,CumPastInflation
0,2008,1.32
1,2009,1.27
2,2010,1.28
3,2011,1.22
4,2012,1.16
5,2013,1.12
6,2014,1.09
7,2015,1.07
8,2016,1.05
9,2017,1.04


**5.4**	Uplift (Past Inflation) Incremental Amounts

Here we will account for past inflation for the incremental amounts, NOT the cumulative amounts.  *Unhide to view output

Code Explanation-

Here for each incremental claim amount, we continually iterate through the "inflation_df" to derive the corresponding inflation year and indexes and subsequently uplift the amount.

Just as before we will first set the inflated incremental amounts equal to the non-inflated for easy referencing.

The code executes 2 nested loops. The first loop is for each incremental claim amount in the "py_data" data-frame, while the second loop is for the each inflation year and inflation index in the "inflation_df" data-frame.

In the first loop, for each incremental claim amount in the "py_data" data-frame, we establish the "Insured Year", "Lag Year" and "Transaction year" (or "Insured Year" plus "Lag Year").

With this ongoing first loop, we then have a second loop to iterate through the "inflation_df" data-frame and establish the "inflation year". While iterating the "inflation_df", we set a conditional that upon reaching a equilibrium point where the respective claim amounts year of valuation (or "Transaction Year") is equal to the inflation year we will execute the proceeding uplift calculation below.  

We will now determine the corresponding "Transaction year" and year-end-cap inflation cumulative index. Finally, divide the latter by the former and multiply it to uplift the incremental claims amount. Note that we divide here as this is a cumulative index. 

Consequently, now we have nominal incremental claims amount with valuation as at the year-end-cap.

For cases where we do not reach the equilibrium point, we will simply do nothing. Hence, the value remains.

In [ ]:
# Uplift (Past Inflation) for Incremental Claims
py_data['Inflated_Claims_Amount'] = py_data['Claims_Amount']

for row in range(0, len(py_data['Insured_Year'])):
    InsuredYear = py_data.loc[row,'Insured_Year']
    LagYear = py_data.loc[row,'Year_Only_Lag']
    TransactionYear = InsuredYear + LagYear
    for year in range(0, len(Inflation_df['Year'])):
        CurrentYearInflation = Inflation_df.loc[year,'Year']
        if  CurrentYearInflation == InsuredYear:
            CurrentYearPerc = Inflation_df.loc[Inflation_df['Year'] == TransactionYear,'CumPastInflation']
            ToYearPerc = Inflation_df.loc[Inflation_df['Year']==YearEndCap,'CumPastInflation'].values[0]
            Uplift = ToYearPerc / CurrentYearPerc
            py_data['Inflated_Claims_Amount'][row] = py_data['Inflated_Claims_Amount'][row]*Uplift
        else:
             py_data['Inflated_Claims_Amount'][row] = py_data['Inflated_Claims_Amount'][row]

print(py_data)

    Insured_Year  Year_Only_Lag  Claims_Amount        cumsum  \
0           2008              0   5.693138e+08  5.693138e+08   
1           2008              1   5.032754e+08  1.072589e+09   
2           2008              2   1.118322e+09  2.190911e+09   
3           2008              3   6.433106e+08  2.834221e+09   
4           2008              4   4.086228e+08  3.242844e+09   
5           2008              5   3.803889e+09  7.046733e+09   
6           2008              6   1.570847e+09  8.617580e+09   
7           2008              7   1.757582e+09  1.037516e+10   
8           2008              8   4.205037e+08  1.079567e+10   
9           2008              9   2.715208e+09  1.351087e+10   
10          2009              0   5.182929e+08  5.182929e+08   
11          2009              1   1.936207e+10  1.988036e+10   
12          2009              2   6.250441e+09  2.613080e+10   
13          2009              3   2.921530e+09  2.905233e+10   
14          2009              4   6.5224

<ipython-input-14-cf3fa23decde>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  py_data['Inflated_Claims_Amount'][row] = py_data['Inflated_Claims_Amount'][row]*Uplift
<ipython-input-14-cf3fa23decde>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  py_data['Inflated_Claims_Amount'][row] = py_data['Inflated_Claims_Amount'][row]


**5.5**	Derive corresponding uplifted Cumulative Amounts

Now we simply use the 'cumsum' function to derive the corresponding inflated cumulative amounts.

In [ ]:
# Get Uplift (Past Inflation) Cumulative Claims
py_data['Inflated_cumsum'] = py_data['Inflated_Claims_Amount'].groupby(py_data['Insured_Year']).cumsum()

**5.6**	Individual Loss Development Factors

Here we will now calculate each development factor (the multiple that resulted in the subsequent years cumulative claim amount) for each insured year. Reason being the IACL underlying assumption is that historical claim trends will follow suit. *Unhide to view output

Code Explanation-

For each row in the "py_data" data-frame, we will retrieve the respective "Insured year", "Lag year", "Transaction year" and "Current cumulative claims amount".

Subsequently, impose a dual 'either or' condition where if the "Transaction year" exceeds the year-end-cap or does not have a proceeding cumulative amount in the next "Lag year" we will have a zero LDF. Reason being this falls into the predicted year range which exceeds our past data range.

Correspondingly, upon not meeting this condition we will derive the "Next cumulative claims amount". We do this by simply looking up the same "Insured year" but adding 1 to the current "Lag year" ("Lag year" plus one).

Finally, we divide the "Next cumulative claims amount" by the "Current cumulative claims amount" to derive the individual LDF and impute it.

In [ ]:
# Inflated
py_data['Inflated_LossDF'] = 1

for row in range(0, len(py_data['Insured_Year'])):
    InsuredYear = py_data.loc[row, 'Insured_Year']
    LagYr = py_data.loc[row, 'Year_Only_Lag']
    CurrentYear = py_data.loc[row, 'Insured_Year'] + py_data.loc[row, 'Year_Only_Lag']
    CurrCumAmt = py_data.loc[row, 'Inflated_cumsum']

    if CurrentYear > YearEndCap or len(py_data.loc[(py_data['Insured_Year'] == InsuredYear) & (
            py_data['Year_Only_Lag'] == (LagYr + 1)), 'Inflated_cumsum']) == 0:
        NextCumAmt = 0
    else:
        NextCumAmt = py_data.loc[(py_data['Insured_Year'] == InsuredYear) & (
                    py_data['Year_Only_Lag'] == (LagYr + 1)), 'Inflated_cumsum'].values[0]

    LDF = NextCumAmt / CurrCumAmt
    py_data.loc[row, 'Inflated_LossDF'] = LDF

print(py_data['Inflated_LossDF'])

0      1.918807
1      2.055717
2      1.309948
3      1.158066
4      2.315983
5      1.241108
6      1.221425
7      1.044199
8      1.275940
9      0.000000
10    38.065535
11     1.329797
12     1.121917
13     1.251273
14     1.191074
15     1.015927
16     1.078034
17     1.095137
18     0.000000
19     8.189267
20     1.830607
21     1.061919
22     1.255354
23     1.197079
24     1.063266
25     4.455149
26     0.000000
27    12.506815
28     1.605898
29     1.382411
30     1.781978
31     1.272632
32     1.147537
33     0.000000
34     9.874797
35     1.147667
36     1.135754
37     1.579504
38     1.236447
39     0.000000
40     2.377169
41     2.421304
42     1.285037
43     1.093010
44     0.000000
45     4.300941
46     1.209511
47     4.094648
48     0.000000
49     5.923891
50     2.872269
51     0.000000
52     1.989408
53     0.000000
54     0.000000
Name: Inflated_LossDF, dtype: float64


In [ ]:
# Non Inflated
py_data['LossDF'] = 1

for row in range(0, len(py_data['Insured_Year'])):
    InsuredYear = py_data.loc[row, 'Insured_Year']
    LagYr = py_data.loc[row, 'Year_Only_Lag']
    CurrentYear = py_data.loc[row, 'Insured_Year'] + py_data.loc[row, 'Year_Only_Lag']
    CurrCumAmt = py_data.loc[row, 'cumsum']

    if CurrentYear > YearEndCap or len(py_data.loc[(py_data['Insured_Year'] == InsuredYear) & (
            py_data['Year_Only_Lag'] == (LagYr + 1)), 'cumsum']) == 0:
        NextCumAmt = 0
    else:
        # .values[0] code to output only values and not entire row
        NextCumAmt = py_data.loc[
            (py_data['Insured_Year'] == InsuredYear) & (py_data['Year_Only_Lag'] == (LagYr + 1)), 'cumsum'].values[0]

    LDF = NextCumAmt / CurrCumAmt
    py_data.loc[row, 'LossDF'] = LDF

print(py_data['LossDF'])

0      1.884004
1      2.042637
2      1.293627
3      1.144175
4      2.173010
5      1.222918
6      1.203953
7      1.040530
8      1.251509
9      0.000000
10    38.357389
11     1.314403
12     1.111804
13     1.224508
14     1.169780
15     1.014145
16     1.068128
17     1.083033
18     0.000000
19     7.852270
20     1.785043
21     1.057946
22     1.233444
23     1.180006
24     1.057525
25     4.128605
26     0.000000
27    11.940906
28     1.582596
29     1.363124
30     1.739227
31     1.259127
32     1.140383
33     0.000000
34     9.568770
35     1.143194
36     1.129732
37     1.546342
38     1.225528
39     0.000000
40     2.340281
41     2.379256
42     1.276232
43     1.089894
44     0.000000
45     4.240373
46     1.204705
47     4.006813
48     0.000000
49     5.831855
50     2.848494
51     0.000000
52     1.979985
53     0.000000
54     0.000000
Name: LossDF, dtype: float64


In [ ]:
py_data.to_csv("py_data.csv", index =True)